##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Serving を使用して TensorFlow モデルをトレーニングしてサービングする

**注意: このノートブックは、Google Colab でのみ実行するように設計されています**。これは root アクセスのあるシステムにパッケージをインストールします。ローカルの Jupyter ノートブックで実行する場合は、注意して続行してください。

注：この例は、Jupyter スタイルのノートブックで今すぐ実行できます。セットアップは必要ありません。「Google Colabで実行」をクリックするだけです

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<tr>
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/serving/rest_simple"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/serving/rest_simple.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/serving/rest_simple.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tfx/tutorials/serving/rest_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</tr>
</table></div>

このガイドでは、ニューラルネットワークモデルをトレーニングし、スニーカーやシャツなどの衣類の[画像](https://github.com/zalandoresearch/fashion-mnist)を分類し、トレーニングしたモデルを保存してから、[TensorFlow Serving](https://www.tensorflow.org/serving/) でサービングします。ここでは TensorFlow でのモデリングとトレーニングではなく、TensorFlow Serving に焦点が当てられているため、モデリングとトレーニングに焦点を当てた完全な例については、[基本的な分類の例](https://github.com/tensorflow/docs/blob/master/site/en/r1/tutorials/keras/basic_classification.ipynb)を参照してください。

このガイドでは、高レベル API である [tf.keras](https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/keras.ipynb) を使用して、TensorFlow でモデルを構築およびトレーニングします。

In [ ]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major == 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

In [ ]:
# TensorFlow and tf.keras
print("Installing dependencies for Colab environment")
!pip install -Uq grpcio==1.26.0

import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))

## モデルの作成

### Fashion-MNIST データセットをインポートする

このガイドでは、[Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) データセットを使用します。このデータセットには、10 カテゴリの 70,000 のグレースケール画像が含まれています。次のように、画像は低解像度（28 x 28ピクセル）で個々の衣料品を示しています。

<table>
  <tr><td>     <img src="https://tensorflow.org/images/fashion-mnist-sprite.png" alt="Fashion MNIST sprite">   </td></tr>
  <tr><td align="center">     <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST サンプル</a> (作成者： Zalando、MIT ライセンス).<br> </td></tr>
</table>

Fashion MNIST は、画像処理のための機械学習での "Hello, World" としてしばしば登場する MNIST データセットの代替として開発されたデータセットです。データをインポートして読み込むだけで、TensorFlow から直接 Fashion MNIST にアクセスできます。

注意：これらは実際には画像ですが、バイナリ画像オブジェクトではなく、NumPy 配列として読み込まれます。

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))

### モデルをトレーニングして評価する

ここではモデリングには焦点を当てていないので、可能な限り単純な CNN を使用します。

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(10, name='Dense')
])
model.summary()

testing = False
epochs = 5

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('\nTest accuracy: {}'.format(test_acc))

## モデルを保存する

トレーニング済みモデルを TensorFlow Serving に読み込むには、最初にモデルを [SavedModel](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/saved_model) 形式で保存する必要があります。これにより、明確に定義されたディレクトリ階層に protobuf ファイルが作成され、バージョン番号が含まれます。[TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving) を使用すると、推論リクエストを行うときに使用するモデルのバージョン、つまり「サービング可能」を選択できます。各バージョンは、指定されたパスの下の異なるサブディレクトリにエクスポートされます。

In [ ]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

## 保存したモデルを調べる

コマンドラインユーティリティ`saved_model_cli`を使用して、SavedModel の [MetaGraphDefs](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/MetaGraphDef)（モデル）と [SignatureDefs](../signature_defs)（呼び出すことができるメソッド）を確認します。TensorFlow ガイドの [SavedModel CLI](https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/saved_model.md#cli-to-inspect-and-execute-savedmodel) に関する説明を参照してください。

In [ ]:
!saved_model_cli show --dir {export_path} --all

これはモデルについて多く情報を与えてくれます。ここでは、モデルをトレーニングしたばかりなので、入力と出力はすでにわかっていますが、知らない場合は、これは重要な情報になります。たとえば、これがグレースケール画像データであることなど、完全な情報は提供されませんが、有用な情報を提供してくれます。

## TensorFlow Serving を使用してモデルをサービングする

**注意: これを Google Colab で実行していない場合は、**以下のセルは root アクセスのあるシステムにパッケージをインストールします。ローカルの Jupyter ノートブックで実行する場合は、注意して続行してください。

### TensorFlow Serving ディストリビューション URI をパッケージソースとして追加する

この Colab は Debian 環境で実行されるため、[Aptitude](https://wiki.debian.org/Aptitude) を使用して TensorFlow Serving をインストールする準備をします。`tensorflow-model-server`パッケージを Aptitude が認識しているパッケージのリストに追加します。root として実行していることに注意してください。

注意：この例では TensorFlow Serving をネイティブで実行していますが、[Docker コンテナで実行することもできます](https://www.tensorflow.org/tfx/serving/docker)。これは TensorFlow Serving を使い始めるのに最も簡単な方法の 1 つです。

In [ ]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''

In [ ]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | {SUDO_IF_NEEDED} tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | {SUDO_IF_NEEDED} apt-key add -
!{SUDO_IF_NEEDED} apt update

### TensorFlow Serving のインストール

必要なのは以下のコマンドライン 1 行だけです。

In [ ]:
!{SUDO_IF_NEEDED} apt-get install tensorflow-model-server

### TensorFlow Serving の実行をはじめる

これから TensorFlow Serving の実行を開始し、モデルを読み込み始めます。読み込み後、REST を使用して推論リクエストの作成を開始できます。以下の重要なパラメータがあります。

- `rest_api_port`: REST リクエストで使用するポート。
- `model_name`: これを REST リクエストの URL で使用します。任意の名前を使用します。
- `model_base_path`: これは、モデルを保存したディレクトリへのパスです。


In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=fashion_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1


In [ ]:
!tail server.log

## TensorFlow Serving でモデルにリクエストを出す

まず、テストデータからランダムな例を見てみましょう。

In [ ]:
def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx].reshape(28,28))
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

import random
rando = random.randint(0,len(test_images)-1)
show(rando, 'An Example Image: {}'.format(class_names[test_labels[rando]]))

興味深いことに気が付きましたか？次に、3 つの推論リクエストのバッチ用の JSON オブジェクトを作成し、モデルがどの程度認識しているかを確認しましょう。

In [ ]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:3].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

### REST リクエストを行う

#### サーバブルの最新バージョン

予測リクエストを POST としてサーバーの REST エンドポイントに送信し、3 つの例を渡します。特定のバージョンを指定せずに、サーバーにサーバブルの最新バージョンを提供するように依頼します。

In [ ]:
# docs_infra: no_execute
!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fashion_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

show(0, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
  class_names[np.argmax(predictions[0])], np.argmax(predictions[0]), class_names[test_labels[0]], test_labels[0]))

#### 特定バージョンのサーバブル

次に、特定のバージョンのサーバブルのを指定します。1 つしかないので、バージョン 1 を選択しましょう。また、3 つの結果すべてを確認します。

In [ ]:
# docs_infra: no_execute
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fashion_model/versions/1:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

for i in range(0,3):
  show(i, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
    class_names[np.argmax(predictions[i])], np.argmax(predictions[i]), class_names[test_labels[i]], test_labels[i]))